### Préparation inférence

In [1]:
import cv2
import torch
import torchvision
from torch2trt import TRTModule
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera

device = torch.device('cuda')
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('RF1_trt.pth'))
car = NvidiaRacecar()
camera = CSICamera(width=224, height=224, capture_fps=65)

### Correcteur Proportionel

In [ ]:
from utils import preprocess
import numpy as np

STEERING_GAIN = -4.00
#STEERING_BIAS = -0.15
STEERING_BIAS = 0

car.throttle = -0.3

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    car.steering = x * STEERING_GAIN + STEERING_BIAS

### Correcteur Proportionnel et dérivateur

In [ ]:
from utils import preprocess
import numpy as np
import time 

#kp = 2.5
kp = 2.25
kd = 8.0
last_x = 0

#car.throttle = -0.4
car.steering_offset = 0

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    car.steering = - (x * kp + ( x - last_x ) *kd)
    car.throttle = -0.25
    last_x = x

### Accélérateur dynamique

In [ ]:
from utils import preprocess
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import asyncio

# Initial parameters
kp = - 1.65
kd = - 12.0
last_x = 0

car.throttle = 0.3
car.steering_offset = 0


# Create a trackbar widget for throttle control
throttle_slider = widgets.FloatSlider(
    value=0.26,  # initial value
    min=-1.0,  # minimum value
    max=1.0,  # maximum value
    step=0.01,  # step size
    description='Throttle:',
    orientation='horizontal'
)

# Display the trackbar
display(throttle_slider)

# Callback function to update car throttle and steering
async def update_car():
    while True:
        global last_x
        image = camera.read()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0])
        car.steering = - (x * kp + ( x - last_x ) *kd)
        last_x = x

        # Update throttle and steering values
        car.throttle = throttle_slider.value
        #car.steering = x * STEERING_GAIN + STEERING_BIAS
        
        await asyncio.sleep(0.001)  # Non-blocking sleep for periodic update

# Start the asynchronous update loop
loop = asyncio.get_event_loop()
loop.create_task(update_car())

# If you are running this in a script, you need to start the event loop
# In a Jupyter Notebook, the loop runs automatically.
if not loop.is_running():
    loop.run_until_complete(update_car())

In [ ]:
from utils import preprocess
import socket
import time
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import asyncio
from threading import Thread, Event, Lock

# Initial parameters
kp = -1.65
kd = -12.0
last_x = 0

car.throttle = 0.3
car.steering_offset = 0

# Variable to track the last received signal time
last_signal_time = time.time()

# Create an event to signal if the robot should stop
stop_event = Event()

# Lock for controlling access to car throttle and steering
car_control_lock = Lock()

# Create a trackbar widget for throttle control
throttle_slider = widgets.FloatSlider(
    value=0.26,  # initial value
    min=-1.0,  # minimum value
    max=1.0,  # maximum value
    step=0.01,  # step size
    description='Throttle:',
    orientation='horizontal'
)

# Display the trackbar
display(throttle_slider)

# Function to check internet connection
'''def check_internet_socket(host="8.8.8.8", port=53, timeout=3):
    try:
        socket.setdefaulttimeout(timeout)
        socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect((host, port))
        print("Internet is up")
        return True
    except socket.error as ex:
        print("Internet is down")
        with car_control_lock:
            car.throttle = 0
        return False
'''

def check_internet_socket(min_speed_mbps=20):
    try:
        st = speedtest.Speedtest()
        st.download()  # Effectue un test de débit descendant
        download_speed_mbps = st.results.download / 1_000_000  # Convertir en Mbps
        print(f"Download speed: {download_speed_mbps:.2f} Mbps")
        if download_speed_mbps >= min_speed_mbps:
            print("Internet speed is sufficient")
            return True
        else:
            print("Internet speed is too low")
            return False
    except Exception as ex:
        print(f"Failed to test internet speed: {ex}")
        return False
# Keep alive function running in a separate thread
def keep_alive():
    print("Keep_alive")
    while not stop_event.is_set():
        if not check_internet_socket():
            print("No internet connection, stopping the robot")
            with car_control_lock:
                car.throttle = 0
            stop_event.set()

# Asynchronous callback function to update car throttle and steering
async def update_car():
    global last_x, last_signal_time
    try:
        while not stop_event.is_set():
            image = camera.read()
            image = preprocess(image).half()
            output = model_trt(image).detach().cpu().numpy().flatten()
            x = float(output[0])
            with car_control_lock:
                car.steering = - (x * kp + (x - last_x) * kd)
                car.throttle = throttle_slider.value
            last_x = x

            last_signal_time = time.time()  # Update last signal time
            await asyncio.sleep(0.001)  # Non-blocking sleep for periodic update
    except asyncio.CancelledError:
        pass
    finally:
        with car_control_lock:
            car.throttle = 0  # Ensure the robot stops if the loop exits
        print("update_car loop has ended, robot stopped")

# Start the keep_alive thread
keep_alive_thread = Thread(target=keep_alive)
keep_alive_thread.start()

# Start the asynchronous update loop
loop = asyncio.get_event_loop()
loop.create_task(update_car())

# If you are running this in a script, you need to start the event loop
# In a Jupyter Notebook, the loop runs automatically.
if not loop.is_running():
    try:
        loop.run_until_complete(update_car())
    except KeyboardInterrupt:
        print("Interrupted by user, stopping the robot")
        stop_event.set()
    finally:
        stop_event.set()
        keep_alive_thread.join()
        with car_control_lock:
            car.throttle = 0  # Ensure the robot stops if the loop exits


### Envoie sur flask

In [ ]:
from utils import preprocess
import numpy as np
import requests
import json

STEERING_GAIN = -4.00
#STEERING_BIAS = -0.15
STEERING_BIAS = 0

#car.throttle = -0.4
car.throttle = -0.0


# Adresse du serveur Flask
url = "http://192.168.0.77:5000/receive_data"

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    
    # Envoyer la valeur x au serveur Flask
    data = {'x': x}
    try:
        response = requests.post(url, json=data)
        if response.status_code == 200:
            print("Data sent successfully")
        else:
            print(f"Failed to send data, status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Error sending data: {e}")
    
    car.steering = x * STEERING_GAIN + STEERING_BIAS


#### Envoie sur serveur puis récupération

In [ ]:
from utils import preprocess
import numpy as np
import requests
import json

STEERING_GAIN = -4.00
#STEERING_BIAS = -0.15
STEERING_BIAS = 0

car.throttle = -0.18

# Adresse du serveur Flask
url = "http://192.168.0.77:5000/receive_data"

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    
    # Envoyer la valeur x au serveur Flask
    data = {'x': x}
    try:
        response = requests.post(url, json=data)
        if response.status_code == 200:
            # Extraire le résultat renvoyé par le serveur
            result = response.json().get('result')
            print(f"Received result from server: {result}")
            
            # Appliquer le résultat à car.steering
            car.steering = result * STEERING_GAIN + STEERING_BIAS
        else:
            print(f"Failed to send data, status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Error sending data: {e}")


In [ ]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import requests
import time
import numpy as np
from utils import preprocess

#kp = 2.5
kp = 2.25
kd = 8.0
last_x = 0
car.throttle = -0.2

car.steering_offset = 0

# Adresse du serveur Flask
url = "http://192.168.0.77:5000/receive_data"

# Configurer une session persistante avec retry pour une meilleure performance
#session = requests.Session()
#retry = Retry(connect=3, backoff_factor=0.5)
#adapter = HTTPAdapter(max_retries=retry)
#session.mount('http://', adapter)


while True:
    a = time.time()*1000
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    
    # Envoyer les variables x et last_x au serveur Flask
    data = {'x': x, 'last_x': last_x}
    try:
        response = requests.post(url, json=data)
        if response.status_code == 200:
            # Extraire le résultat renvoyé par le serveur
            result = response.json().get('result')
           # print(f"Received result from server: {result}")
            
            # Appliquer le résultat à car.steering
            car.steering = result
            print("correcteur PD")
        else:
           # print(f"Failed to send data, status code: {response.status_code}")
            car.steering = -(x * kp )
            print("correcteur P")
    except requests.exceptions.RequestException as e:
       # print(f"Error sending data: {e}")
        car.steering = -(x * kp )
        print("correcteur P")
    
    # Mettre à jour last_x
    last_x = x
   # print(f"temps de boucle : {time.time()*1000 - a} ms")
    # Contrôle du véhicule
  


### test avec boucle évènementiel pour changer de modèle

In [2]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import requests
import time
import socket
import threading
import numpy as np
from utils import preprocess

kp = 2.25
kd = 8.0
last_x = 0
car.throttle = -0.2
car.steering_offset = 0

# Adresse du serveur Flask
url = "http://192.168.0.77:5000/receive_data"

# Variable pour stocker l'état de la connexion Internet
internet_connected = False

# Fonction pour vérifier la connexion Internet
def check_internet_socket(host="8.8.8.8", port=53, timeout=3):
    global internet_connected
    while True:
        try:
            socket.setdefaulttimeout(timeout)
            socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect((host, port))
            if not internet_connected:
                print("Internet is up")
            internet_connected = True
        except socket.error as ex:
            if internet_connected:
                print("Internet is down")
            internet_connected = False
        time.sleep(0.0001)  # Vérifie la connexion toutes les 10 secondes

def update_car(change):
    global last_x

    # Lire l'image de la caméra et la prétraiter
    image = change['new']
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])

    if internet_connected:
        # Internet est disponible, essayer de communiquer avec le serveur
        data = {'x': x, 'last_x': last_x}
        try:
            response = requests.post(url, json=data)
            if response.status_code == 200:
                # Extraire le résultat renvoyé par le serveur
                result = response.json().get('result')
                car.steering = result
                print("Correcteur PD avec serveur")
            else:
                car.steering = -(x * kp)
                print("Correcteur P local (échec de la communication avec le serveur)")
        except requests.exceptions.RequestException as e:
            car.steering = -(x * kp)
            print("Correcteur P local (erreur de communication)")
    else:
        # Pas d'Internet, utiliser le correcteur proportionnel local
        car.steering = -(x * kp)
        print("Correcteur P local (pas d'internet)")

    # Mettre à jour last_x
    last_x = x

# Thread pour surveiller la connexion Internet
internet_thread = threading.Thread(target=check_internet_socket)
internet_thread.daemon = True
internet_thread.start()

# Initialiser le processus de mise à jour
update_car({'new': camera.value})  # Appel initial pour initialiser
camera.observe(update_car, names='value')  # Attacher la fonction de mise à jour à la caméra


/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Internet is up
Correcteur PD avec serveur
